In [ ]:
# default_exp markdown

# Markdown

> Conversion to markdown capabilities.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import string
import functools
from typing import Union, List

# to avoid tqdm's experimental warning
import warnings
warnings.filterwarnings("ignore", message='Using `tqdm.autonotebook.tqdm` in notebook mode')

import gift_wrapper.question
import gift_wrapper.latex
import gift_wrapper.parsing
import gift_wrapper.transformer

import py2gift.util

## Creating a header in markdown

In [ ]:
# export
def markdown_header(
        text: str,
        template: string.Template = string.Template('<span style="font-family:Papyrus; font-size:2em;">$text</span>')
    ) -> str:
    """
    Returns a markdown header for a given text.

    **Parameters**
    
    - `text` : str
        
        Text for the header.
    
    - `template` : string template
        
        The template that defines the appearance of the header, and in which `text` will be embedded.

    **Returns**
    
    - `out` : str
        Markdown-compatible text.

    """

    return f'\n{template.substitute(text=text)}\n\n'

In [ ]:
py2gift.util.render_latex(markdown_header('Header 1'))


<span style="font-family:Papyrus; font-size:2em;">Header 1</span>



## Markdown from user settings for a question

An example **numerical** question...

In [ ]:
question_settings = {
    'name': 'Q1',
    'class': 'Numerical',
    'statement': 'Consider the constellation\nsample_images/1D_3_elements.svg',
    'solution': {
        'value': 42,
        'error': '10%'
    },
    'feedback': 'We need to...'
}
class_name = gift_wrapper.question.user_settings_to_class_init(question_settings)
numerical_question = getattr(gift_wrapper.question, class_name)(**question_settings)

...and one of **multiple choice**

In [ ]:
question_settings = {
    'name': 'Q2',
    'class': 'MultipleChoice',
    'statement': 'Choose...',
    'answers':{
        'perfect': 'yep',
        'wrong': ['foo', 'fa']
    },
    'feedback': 'We need to...'
}
class_name = gift_wrapper.question.user_settings_to_class_init(question_settings)
multiple_choice_question = getattr(gift_wrapper.question, class_name)(**question_settings)

## Patching gift-wrapper questions

### Base (abstract) *HtmlQuestion*

In [ ]:
# export
def html_question_to_markdown(self):

    statement = self.statement

    # if a `time` estimate was passed...
    if self.time:

        # ...it is appended at the end of the statement
        statement += f'\n\n\n*Estimated time: {self.time} minutes*\n'

    feedback = (f'{markdown_header("Feedback")}' + self.feedback.rstrip()) if self.feedback else ''

    # a copy of each list is made so that the class attribute is not modified
#     latex_in_text_substitutions = [e.copy() for e in gift_wrapper.parsing.latex_in_text_substitutions[:2]]
    latex_in_text_substitutions = [e.copy() for e in gift_wrapper.transformer.LatexCommandsWithinText.patterns[:2]]

    # \textbf-related substitutions are tweaked
    latex_in_text_substitutions[0][1] = r'**\1**'
    latex_in_text_substitutions[0][2] = r'\*\*([^\*]+)\*\*'

    # \textit-related substitutions are tweaked
    latex_in_text_substitutions[1][1] = r'*\1*'
    latex_in_text_substitutions[1][2] = r'\*([^\*]+)\*'

    # just like LaTeX, in markdown a single "newline" doesn't reflect in the output, and hence every new line
    # is duplicated
    latex_in_text_substitutions += [['\n', '\n\n', '\n\n', '\n']]

    def apply_substitutions(text: str, substitutions: list):

        res = text

        for s in substitutions:

            res = gift_wrapper.latex.replace_and_replace_only_in_formulas(*s, res)

        return res

    statement = apply_substitutions(statement, latex_in_text_substitutions)
    feedback = apply_substitutions(feedback, latex_in_text_substitutions)

    return f'{markdown_header("Statement")}{statement}\n{feedback}\n'

gift_wrapper.question.HtmlQuestion.to_markdown = html_question_to_markdown

In [ ]:
py2gift.util.render_latex(html_question_to_markdown(numerical_question))


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

Consider the constellation

sample_images/1D_3_elements.svg


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



We need to...


### Numerical question

In [ ]:
# export
def numerical_to_markdown(self):

    res = super(gift_wrapper.question.Numerical, self).to_markdown()

    return res + f'{markdown_header("Solution")} {self.solution_value} (error: {self.solution_error[1:]})\n'

gift_wrapper.question.Numerical.to_markdown = numerical_to_markdown

In [ ]:
py2gift.util.render_latex(numerical_question.to_markdown())


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

Consider the constellation

sample_images/1D_3_elements.svg


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



We need to...

<span style="font-family:Papyrus; font-size:2em;">Solution</span>

 42 (error: .2)


### Multiple-choice question

In [ ]:
# export
def multiple_choice_to_markdown(self):

    res = super(gift_wrapper.question.MultipleChoice, self).to_markdown()

    res += markdown_header('Choices')

    if 'perfect' in self.answers:

        # res += f'---\n'

        # res += f'* {self.answers["perfect"]}\n'
        res += f'* {self.template_wrong_answers.substitute(color="green", text=self.answers["perfect"])}\n'

        # res += f'---\n'

    for a in self.answers['wrong']:

        if isinstance(a, list):

            formatted_grade = self.template_wrong_answers.substitute(
                color='green' if float(a[1]) > 0 else 'red', text=f'{a[1]}%')

            res += f'* {a[0]} (**{formatted_grade}**)\n'
            # res += f'* {a[0]} (**{a[1]}%**)\n'

        else:

            # res += f'* {a}\n'
            res += f'* {self.template_wrong_answers.substitute(color="red", text=a)}\n'

    return res

gift_wrapper.question.MultipleChoice.to_markdown = multiple_choice_to_markdown

In [ ]:
py2gift.util.render_latex(multiple_choice_question.to_markdown())


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

Choose...


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



We need to...

<span style="font-family:Papyrus; font-size:2em;">Choices</span>

* **<font color='green'>yep</font>**
* **<font color='red'>foo</font>**
* **<font color='red'>fa</font>**


## A *gift-wrapper* processor to reformat paths for markdown

In [ ]:
# export
class SvgToMarkdown(gift_wrapper.transformer.Transformer):
    """
    Processor to reformat svg files for including them in markdown strings.
    """

    def __init__(self):

        super().__init__()

        def process_match(f):

            pass

        self.function = functools.partial(
            gift_wrapper.transformer.process_paths, pattern=gift_wrapper.parsing.svg_file,
            process_match=process_match, replacement=r'![](' + r'\1)')

In [ ]:
# export
def from_question(
    q: gift_wrapper.question.HtmlQuestion, processors: List[gift_wrapper.transformer.Transformer]) -> str:
    """
    Returns the markdown representation of a question.

    **Parameters**
    
    - `q` : gift_wrapper.question.HtmlQuestion
        
        The question object.
    
    - `processors` : list
        
        Processors to be applied.

    **Returns**
    
    - `out` : str
        Markdown-compatible text.

    """
    
    markdown = q.to_markdown()

    for f in processors:

        markdown = f(markdown)

    return markdown

In [ ]:
# export
pre_processors = [
    gift_wrapper.transformer.TexToSvg({'already compiled': set(), 'already transferred': set()}), SvgToMarkdown()]

In [ ]:
py2gift.util.render_latex(from_question(numerical_question, pre_processors))


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

Consider the constellation

![](sample_images/1D_3_elements.svg)


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



We need to...

<span style="font-family:Papyrus; font-size:2em;">Solution</span>

 42 (error: .2)


In [ ]:
# export
def settings_to_markdown(settings: dict) -> str:
    """
    Returns a markdown representation of the question given the corresponding settings.

    **Parameters**
    
    - `settings` : dict
        
        User settings (usually read from a YAML file).

    **Returns**
    
    - `markdown`: str
        
        Markdown representation

    """

    class_name = gift_wrapper.question.user_settings_to_class_init(settings, 'unnamed')
    cls = getattr(gift_wrapper.question, class_name)

    return from_question(cls(**settings), pre_processors)

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('80_markdown.ipynb')

Converted 80_markdown.ipynb.
